In [145]:
# coding: utf-8

In [4]:
import pandas as pd
import sqlite3
import re
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy.histories import HistoryClient

In [5]:
cd ../galaxy

/home/u/abc/galaxy


## config

In [6]:
with open('./config/galaxy.ini') as file:
    # default
    db_path = 'database/universe.sqlite'

    # if different value in config file
    for line in file:
        if re.match('^database_connection\ =.*', line):
            db_path = re.sub('database_connection\ = sqlite:///./', '', line)
            # without the index the string finishes with a space
            db_path = re.sub('\?.*', '', db_path)[:-1]

## connect to database

In [7]:
conn = sqlite3.connect(db_path)

In [8]:
api_key = open('../galaxy_api_key').read()
galaxy_url = 'http://localhost:8080'
gi = GalaxyInstance(galaxy_url, api_key)

## alternative: load workflows from local files

## alternative: load histories from files and convert histories to workflows

download histories to a binary file

for id in [item['id'] for item in hi.get_histories()]:
    hi.download_history(id, hi.export_history(id), open(str(id) + '.history', 'w'), chunk_size=4096)

upload binary history to database


## clear workflow table

Display existing workflows

## alternative: upload histories and workflows downloaded with SQL

**ACHTUNG: the following _update_ query may break the versionning of the database**

## Extract workflows from histories

Display existing histories

The attempt to pass a string variable to the curl command in os.system fails (too many levels of \").

The following function stores the _payload_ in a file before passing the query.

In [9]:
def extract_workflow_from_history(wid):
    payload = "{\"from_history_id\": \"" + wid + "\",  \"workflow_name\": \"" + "extracted-" +  wid + "\"}"
    open('payload.json', 'w').write(payload)
    !curl http://localhost:8080/api/workflows?key=$(cat ../galaxy_api_key) -H "Content-Type: application/json"\
    --data @payload.json
    !rm -f payload.json

Extract history and display the extracted history

In [10]:
for item in gi.histories.get_histories():
    extract_workflow_from_history(item['id'])
    print

{"name": "extracted-1cd8e2f6b131e891", "tags": [], "deleted": false, "latest_workflow_uuid": "068cd09c-a58c-4cf4-8fe0-2543d3f6cf58", "url": "/api/workflows/c385e49b9fe1853c", "published": false, "model_class": "StoredWorkflow", "id": "c385e49b9fe1853c"}
{"name": "extracted-f597429621d6eb2b", "tags": [], "deleted": false, "latest_workflow_uuid": "324393fb-e4fb-4053-820b-6f0cedabfb5f", "url": "/api/workflows/f7bb1edd6b95db62", "published": false, "model_class": "StoredWorkflow", "id": "f7bb1edd6b95db62"}


Display uploaded workflows and extracted workflows

## get connections

1. get connections (by id of workflow steps)
2. get tool id corresponding to each workflow step

In [11]:
get_associations_query = '''
SELECT distinct w_s.workflow_id, w_s.id AS out_port, w_c.input_step_id AS in_port
FROM workflow_step AS w_s, workflow_step_connection AS w_c
WHERE w_s.id=w_c.output_step_id
'''
assoc = pd.read_sql(get_associations_query, con=conn)

print assoc[:10]

   workflow_id  out_port  in_port
0           28         1        3


In [12]:
get_tool_id_query = '''
SELECT distinct w_s.id, w_s.tool_id
FROM workflow_step AS w_s
'''

step_tool = pd.read_sql(get_tool_id_query, con=conn)
print step_tool[:10]

   id tool_id
0   1    None
1   2    None
2   3    cat1


## replace degenerated step id by unique tool id

In [13]:
out_tool_ids = [step_tool.tool_id[step_tool.id == item].values[0] for index, item in enumerate(assoc.out_port)]
in_tool_ids = [step_tool.tool_id[step_tool.id == item].values[0] for index, item in enumerate(assoc.in_port)]

assoc.insert(0, 'out_port_tool', pd.DataFrame(out_tool_ids))
assoc.insert(1, 'in_port_tool', pd.DataFrame(in_tool_ids))
print assoc[:10]

  out_port_tool in_port_tool  workflow_id  out_port  in_port
0          None         cat1           28         1        3


## association rule identification

dictionnary with tool as key and tool connected as output as values

## count associations

In [14]:
d = {}

for item in assoc.out_port_tool.unique():
    d[item] = [{}]

for index, tool in enumerate(assoc.out_port_tool):
    #print index, tool, assoc.in_port_tool[index]
    if assoc.in_port_tool[index] not in d[tool][0].keys():
        d[tool][0][assoc.in_port_tool[index]] = 1
    else:
        d[tool][0][assoc.in_port_tool[index]] += 1

## sort associations by frequency

In [15]:
recomm = {'tool': [], 'suggestion': [], 'proof': []}

for tool in d.keys():
    #d[tool].sort(key=lambda x: x[1])çç
    
    tool_max = max(d[None][0], key=d[None][0].get)
    
    recomm['tool'].append(tool)
    recomm['suggestion'].append(tool_max)
    recomm['proof'].append( sorted(d[tool][0].values())[-1]  )

In [16]:
recommendation = pd.DataFrame(recomm)

In [17]:
recommendation.to_hdf('recommendation_table.hdf', 'w')

/home/u/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['suggestion', 'tool']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


## Use 

see [use-suggestions.ipynb](use-suggestions.ipynb) and [use-suggestions.py](use-suggestions.py)